In [21]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)

In [2]:
hs = pd.read_excel('../data/hs_table_vietnamese.xlsx')
hs.dropna(how='all', inplace=True)

In [3]:
# Create section columns
hs.loc[(hs["CONTENT"].str.lower().str.startswith('section')) & (hs["CONTENT"].str.len() < 20), "SECTION"] = hs.loc[(hs["CONTENT"].str.lower().str.startswith('section')) & (hs["CONTENT"].str.len() < 15), "CONTENT"].str[8:]
hs['TITLE'] = hs['SECTION'].shift(1)
hs.loc[pd.notna(hs['TITLE']), 'TYPE'] = 'SECTION TITLE'
del hs['TITLE']

In [4]:
# Create Chapter columns
hs.loc[(hs["CONTENT"].str.lower().str.startswith('chapter')) & (hs["CONTENT"].str.len() < 12), 'CHAPTER'] = hs.loc[(hs["CONTENT"].str.lower().str.startswith('chapter')) & (hs["CONTENT"].str.len() < 12), 'CONTENT'].str[8:]
hs.loc[pd.notna(hs['CHAPTER']), 'TYPE'] = 'CHAPTER NAME'
hs['TEMP'] = hs['CHAPTER'].shift(1)
hs.loc[pd.notna(hs['TEMP']) & pd.isna(hs['TYPE']), 'TYPE'] = 'CHAPTER TITLE'
del hs['TEMP']

In [5]:
# CREATE HS
hs['HS'] = hs['HS'].astype(str)
hs.loc[hs['HS'].str.len() ==  8, 'TYPE'] = 'HS'

In [6]:
# HEADING AND NOTE OF HEADING
hs.loc[(hs['HS'].str.len() == 4) | ((hs['HS'].str.len() == 8) & (hs['HS'].str.endswith('0000'))), 'HEADING'] = hs.loc[(hs['HS'].str.len() == 4) | ((hs['HS'].str.len() == 8) & (hs['HS'].str.endswith('0000'))), 'HS'].str[:4]
hs.loc[pd.notna(hs['HEADING']), 'TYPE'] = 'HEADING NAME'
hs['TEMP'] = hs['HEADING'].shift(1)
hs.loc[pd.notna(hs['TEMP']) & pd.isna(hs['TYPE']), 'TYPE'] = 'NOTE OF HEADING'

In [7]:
# SUBHEADING
hs.loc[hs['HS'].str.len() == 6, 'SUBHEADING'] = hs.loc[hs['HS'].str.len() == 6, 'HS']
hs.loc[pd.notna(hs['SUBHEADING']) & pd.isna(hs['TYPE']), 'TYPE'] = 'SUBHEADING'

In [8]:



del hs['TEMP']

In [9]:
hs.drop(hs[hs['HS'].str.len() > 8].index, inplace=True)
index_to_delete = hs[(~hs['HS'].str.contains('[0-9]', regex=True, na=True)) & (hs['HS'].str.len() > 3)].index
hs.drop(index_to_delete, inplace=True)


In [10]:
hs.loc[pd.notna(hs['SECTION']), 'TYPE'] = 'SECTION NAME'

In [11]:
hs['TEMP'] = hs['SECTION'].shift(2)
hs.loc[pd.notna(hs['TEMP']) & pd.isna(hs['TYPE']), 'TYPE'] = 'NOTE OF SECTION'
del hs['TEMP']

In [12]:
hs['TEMP'] = hs['CHAPTER'].shift(2)
hs.loc[pd.notna(hs['TEMP']) & pd.isna(hs['TYPE']), 'TYPE'] = 'NOTE OF CHAPTER'
del hs['TEMP']

In [13]:
# DEFINE NOTE OF HS
hs.loc[hs['TYPE'] == 'HS', 'TEMP'] = 'AWAIT'
del hs['DESCRIPTION']
hs['TEMP'] = hs['TEMP'].shift(1)
hs.loc[pd.isna(hs['TYPE']) & pd.notna(hs['TEMP']), 'TYPE'] = 'NOTE OF HS'
del hs['TEMP']



In [14]:
# DEFINE NOTE OF SUBHEADING
hs.loc[hs['TYPE'] == 'SUBHEADING', 'TEMP'] = 'AWAIT'
hs['TEMP'] = hs['TEMP'].shift(1)
hs.loc[pd.isna(hs['TYPE']) & pd.notna(hs['TEMP']), 'TYPE'] = 'NOTE OF SUBHEADING'
del hs['TEMP']

In [15]:

del hs['HEADING']
del hs['SUBHEADING']
del hs['NOTE']

In [16]:
hs.ffill(inplace=True)

In [17]:
hs.drop(hs[((hs['CONTENT'] == 'Notes.') | (hs['CONTENT'] == 'Note.'))].index, inplace=True)



In [18]:
hs['HS'].replace('nan', np.NaN, inplace=True)
hs['HS'].replace(u'\xa0', np.NaN, inplace=True)
hs.ffill(inplace=True)

In [19]:
# hs.to_excel('../data/temp.xlsx')
hs['HEADING'] = hs['HS'].str[:4]


In [20]:
hs_dic = []
for _, row in hs.iterrows():
    if row['TYPE'] == 'SECTION NAME':
        hs_dic.append({
            'id': row['SECTION']
        })

    if row['TYPE'] == 'SECTION TITLE':
        hs_dic[-1]['title'] = row['CONTENT']

    # if row['TYPE'] == 'NOTE OF SECTION':
    #     if 'notes' not in hs_dic[-1]:
    #         hs_dic[-1]['notes'] =  []
    #     hs_dic[-1]['notes'].append(row['CONTENT'])


    if row['TYPE'] == 'CHAPTER TITLE':
        if 'chapters' not in hs_dic[-1]:
            hs_dic[-1]['chapters'] =  []
        hs_dic[-1]['chapters'].append({
            'id': row['CHAPTER']
        })
        hs_dic[-1]['chapters'][-1]['title'] = row['CONTENT']

    # if row['TYPE'] == 'NOTE OF CHAPTER':
    #     if 'notes' not in hs_dic[-1]['chapters'][-1]:
    #         hs_dic[-1]['chapters'][-1]['notes'] =  []
    #     hs_dic[-1]['chapters'][-1]['notes'].append(row['CONTENT'])

    if row['TYPE'] == 'HEADING NAME':
        if 'headings' not in hs_dic[-1]['chapters'][-1]:
            hs_dic[-1]['chapters'][-1]['headings'] =  []
        hs_dic[-1]['chapters'][-1]['headings'].append({
            'id': row['HEADING'],
            'title': row['CONTENT']
        })

    # if row['TYPE'] == 'NOTE OF HEADING':
    #     if 'notes' not in hs_dic[-1]['chapters'][-1]['headings'][-1]:
    #         hs_dic[-1]['chapters'][-1]['headings'][-1]['notes'] = []
    #     hs_dic[-1]['chapters'][-1]['headings'][-1]['notes'].append(row['CONTENT'])
    
    
    if row['TYPE'] == 'HS':
        if 'hs' not in hs_dic[-1]['chapters'][-1]['headings'][-1]:
            hs_dic[-1]['chapters'][-1]['headings'][-1]['hs'] =  []
        hs_dic[-1]['chapters'][-1]['headings'][-1]['hs'].append({
            'id': row['HS'],
            'title': row['CONTENT']
        })
    
    # if row['TYPE'] == 'NOTE OF HS':
    #     if 'notes' not in hs_dic[-1]['chapters'][-1]['headings'][-1]['hs'][-1]:
    #         hs_dic[-1]['chapters'][-1]['headings'][-1]['hs'][-1]['notes'] = []
    #     hs_dic[-1]['chapters'][-1]['headings'][-1]['hs'][-1]['notes'].append(row['CONTENT'])

    # if row['TYPE'] == 'SUBHEADING':
    #     if 'hs' not in hs_dic[-1]['chapters'][-1]['headings'][-1]:
    #         hs_dic[-1]['chapters'][-1]['headings'][-1]['hs'] =  []
    #     hs_dic[-1]['chapters'][-1]['headings'][-1]['hs'].append({
    #         'id': row['HS'],
    #         'title': row['CONTENT']
    #     })
    
    # if row['TYPE'] == 'NOTE OF SUBHEADING':
    #     if 'notes' not in hs_dic[-1]['chapters'][-1]['headings'][-1]['hs'][-1]:
    #         hs_dic[-1]['chapters'][-1]['headings'][-1]['hs'][-1]['notes'] = []
    #     hs_dic[-1]['chapters'][-1]['headings'][-1]['hs'][-1]['notes'].append(row['CONTENT'])



import json
with open('data.json', 'w') as f:
    json.dump(hs_dic, f)

In [39]:
with open('vi_data2.json') as json_file:
    data = json.load(json_file)

data.pop()


{'id': 'XXI',
 'title': 'CÁC TÁC PHẨM NGHỆ THUẬT, ĐỒ SƯU TẦM VÀ ĐỒ CỔ',
 'notes': [],
 'chapters': [{'id': '97',
   'title': 'Các tác phẩm nghệ thuật, đồ sưu tầm và đồ cổ',
   'notes': ['Chú giải.',
    '1.- Chương này không bao gồm:',
    '(a) Tem bưu chính hoặc tem thuế chưa sử dụng, bưu thiếp (đã đóng sẵn tem) hoặc loại',
    'tương tự, thuộc nhóm 49.07;',
    '(b) Phông sân khấu, phông trường quay hoặc loại tương tự, bằng vải bạt đã sơn vẽ (nhóm',
    '59.07) trừ khi chúng có thể phân loại được vào nhóm 97.06; hoặc',
    '(c) Ngọc trai, tự nhiên hoặc nuôi cấy, hoặc đá quý hoặc đá bán quý (thuộc các nhóm từ',
    '71.01 đến 71.03).',
    '2.- Theo nội dung của nhóm 97.02, khái niệm “bản khắc, bản in và bản in lytô nguyên bản”',
    'có nghĩa là các bản in được in trực tiếp, đen và trắng hoặc in màu, bằng một hoặc một vài',
    'bản kẽm do các nghệ nhân thực hiện hoàn toàn bằng tay, không phân biệt quy trình hoặc',
    'loại nguyên liệu mà họ sử dụng, nhưng không bao gồm quá trình cơ

In [47]:
from __future__ import print_function
import json
import sys

# Tree in JSON format
s = '{"Harry": {"children": ["Bill", {"Jane": {"children": [{"Diane": {"children": ["Mary"]}}, "Mark"]}}]}}'

# Convert JSON tree to a Python dict
data = json.loads(s)

# Convert back to JSON & print to stderr so we can verify that the tree is correct.
# print(json.dumps(data, indent=4), file=sys.stderr)

# Extract tree edges from the dict
edges = []

def get_edges(treedict, parent=None):
    name = next(iter(treedict.keys()))
    if parent is not None:
        edges.append((parent, name))
    for item in treedict[name]["children"]:
        if isinstance(item, dict):
            get_edges(item, parent=name)
        else:
            edges.append((name, item))

get_edges(data)

# Dump edge list in Graphviz DOT format
print('strict digraph tree {')
for row in edges:
    print('    {0} -> {1};'.format(*row))
print('}')

strict digraph tree {
    Harry -> Bill;
    Harry -> Jane;
    Jane -> Diane;
    Diane -> Mary;
    Jane -> Mark;
}


{
    "Harry": {
        "children": [
            "Bill",
            {
                "Jane": {
                    "children": [
                        {
                            "Diane": {
                                "children": [
                                    "Mary"
                                ]
                            }
                        },
                        "Mark"
                    ]
                }
            }
        ]
    }
}
